In [1]:
# useing this variable for saving models and summaries
exp_name = "DWN9004_opp"

## Importing Libraries

In [2]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
import numpy as np
from hdf5storage import savemat
import hickle 

import sys
sys.path.append('../src')

In [3]:
import models.wavelet_conv_model as wavelet_conv_model
import training
import evaluation.nn_eval as nn_eval

## Loading synthetic data

In [4]:
#loading data
data_path= '../data/opp4.hkl'
with open(data_path, 'r') as fin:
    data = hickle.load(fin)
    
train_data = data['train_data']
test_data = data['test_data']
validation_data = data['validation_data']

train_gt = data['train_gt']
validation_gt = data['validation_gt']
test_gt = data['test_gt']

#index_of_changes = data['index_of_changes_test']
 

In [5]:
train_gt.shape, train_data.shape

((1, 700165, 1), (1, 700165, 77))

## Model

In [6]:
# Model parameters
n_variables = 77
learning_rate = 0.001
batch_size = 1

wavelet_levels = 5    #number of wavelet layer to have in the network
convolution_levels = 3    #number of convolution to have in the network
pooling_stride = [4, 4, 4]
weights_shape = [[9, n_variables, 32],
                 [5, 32, 32],
                 [5, 32, 32]]
bias_shape = [32, 32, 32]
wavelet_weights_shape = [[5, n_variables, n_variables]]    #high_pass and low_pass dimensions
activation = tf.nn.relu
n_classes = 1

##positive weight for weighted cross entropy
''' s = train_data.shape
resolution = np.prod(pooling_stride)
gt_effective = np.reshape(train_gt, (s[0]*(s[1]/resolution), resolution))
pos_weight = (1.0 - gt_effective.max(axis=1).mean()) / gt_effective.max(axis=1).mean()
'''

s = train_gt.shape
resolution = np.prod(pooling_stride)
pooled_gt = np.reshape(train_gt[0,:(s[0]*s[1]/resolution)*resolution,:], (s[0]*(s[1]/resolution), resolution, s[2])).max(axis=1)
pos_weight = (1.0 - pooled_gt.mean(axis = 0)) / pooled_gt.mean(axis=0)
pos_weight = pos_weight[0]

In [7]:
train_data = train_data[:,:-(train_data.shape[1]%((2**(wavelet_levels-1))*resolution)),:]
validation_data = validation_data[:,:-(validation_data.shape[1]%((2**(wavelet_levels-1))*resolution)),:]
test_data = test_data[:,:-(test_data.shape[1]%((2**(wavelet_levels-1))*resolution)),:]

train_gt = train_gt[:,:-(train_gt.shape[1]%((2**(wavelet_levels-1))*resolution)),:]
validation_gt = validation_gt[:,:-(validation_gt.shape[1]%((2**(wavelet_levels-1))*resolution)),:]
test_gt = test_gt[:,:-(test_gt.shape[1]%((2**(wavelet_levels-1))*resolution)),:]


In [8]:
print pos_weight

2.17746


In [9]:
# Launch the graph
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [10]:
summary_path = '../summary/' + exp_name 
checkpoint_path = '../model/' + exp_name + '/checkpoint' 

In [11]:
model = wavelet_conv_model.Model(n_variables = n_variables,
                                 learning_rate = learning_rate,
                                 n_classes = n_classes)

In [12]:
model.build_neural_wavelet_layer(pos_weight = pos_weight, 
                                 learning_rate = learning_rate,
                                 wavelet_levels = wavelet_levels,    
                                 convolution_levels = convolution_levels,    
                                 pooling_stride = pooling_stride,     
                                 weights_shape = weights_shape,
                                 bias_shape = bias_shape,
                                 wavelet_weights_shape = wavelet_weights_shape,    
                                 activation = activation,
                                 trainable_wavelet=False,
                                 trainable_conv=False,
                                 trainable_last=True,
                                )
                        

In [13]:
trainer = training.Trainer(model = model,
                    data_train = train_data,
                    ground_truth_train = train_gt,
                    data_validation = test_data,
                    ground_truth_validation = test_gt,
                    sess = sess,
                    summary_path = summary_path,
                    batch_size = batch_size,
                    global_step = 0)

In [14]:
#load the model
saver = tf.train.Saver(var_list = [
    model.weights['wc0'],model.weights['wc1'],model.weights['wc2'],
    model.bias['bc0'],model.bias['bc1'],model.bias['bc2'],
    model.weights['high_pass'],model.weights['low_pass']
])
saver.restore(sess, '../model/DWN9001_opp/checkpoint-3474')

INFO:tensorflow:Restoring parameters from ../model/DWN9001_opp/checkpoint-3474


In [ ]:
trainer.train(max_iter = 50000,
                train_eval_step = 1, 
                validation_eval_step = 1,
                display_step = 1)

Train Batch Evaluation
AUC = 0.414 vs. 0.314
F1 = 0.480 vs. 0.479
Precision = 0.316 vs. 0.315
Recall = 1.000 vs. 1.000
Validation
AUC = 0.251 vs. 0.180
F1 = 0.334 vs. 0.320
Precision = 0.223 vs. 0.191
Recall = 0.664 vs. 1.000
Minibatch Loss= 6.117275
Train Batch Evaluation
AUC = 0.417 vs. 0.320
F1 = 0.480 vs. 0.479
Precision = 0.316 vs. 0.315
Recall = 1.000 vs. 1.000
Validation
AUC = 0.251 vs. 0.200
F1 = 0.338 vs. 0.323
Precision = 0.226 vs. 0.196
Recall = 0.678 vs. 0.913
Minibatch Loss= 5.814163
Train Batch Evaluation
AUC = 0.418 vs. 0.319
F1 = 0.480 vs. 0.479
Precision = 0.316 vs. 0.316
Recall = 1.000 vs. 0.995
Validation
AUC = 0.259 vs. 0.195
F1 = 0.338 vs. 0.323
Precision = 0.226 vs. 0.194
Recall = 0.675 vs. 0.978
Minibatch Loss= 5.539975
Train Batch Evaluation
AUC = 0.417 vs. 0.314
F1 = 0.480 vs. 0.479
Precision = 0.316 vs. 0.315
Recall = 1.000 vs. 1.000
Validation
AUC = 0.256 vs. 0.199
F1 = 0.339 vs. 0.322
Precision = 0.224 vs. 0.192
Recall = 0.697 vs. 0.997
Minibatch Loss= 5.296

In [ ]:
#Save the model
saver = tf.train.Saver()
saver.save(sess, checkpoint_path, global_step=trainer.global_step)

In [ ]:
#load the model
saver = tf.train.Saver()
saver.restore(sess, "../models/model" + exp_name + "/model")

In [ ]:
#This is the model evaluation on test dataset
probability_of_pos= nn_eval.evaluate_model (model, sess, test_data, test_gt)

In [ ]:
#shape of probability_change is (batch size, T, num_classes)
#since num_classes = 1 we squeeze that axis
probability_change = np.squeeze(probability_of_pos, axis = 2)

In [ ]:
savemat('../poc/poc'+exp_name+'.mat', {'poc':probability_change})